In [4]:
# --- Passo 1.1: Instalar a biblioteca requests ---

!pip install requests

# --- Passo 1.2: Configurar a API Key do Google Gemini ---

import os
import requests # Importa a biblioteca requests
import json # formatar os dados da requisição e resposta

# Importa ferramentas do Colab para carregar secrets ou pedir input seguro
from google.colab import userdata
import getpass # Para input seguro se não usar secrets

try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab.")
except userdata.NotInUserDatabaseError:

    print("API Key não encontrada nos Secrets do Colab.")
    print("Por favor, cole sua API Key abaixo:")
    API_KEY = getpass.getpass("Digite sua API Key: ")

if not API_KEY:
    print("Erro: API Key não foi fornecida. Não é possível continuar.")
    # adicionar um 'raise SystemExit(1)' ou 'exit(1)' para parar a execução
else:
    print("API Key configurada com sucesso!")

API Key carregada dos Secrets do Colab.
API Key configurada com sucesso!


In [5]:
# --- Passo 2: Definir Endpoint da API e Headers ---

API_BASE_URL = "https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent"

# Os cabeçalhos da requisição HTTP, incluindo a API Key
HEADERS = {
    "Content-Type": "application/json",
    "x-goog-api-key": API_KEY
}

print(f"Endpoint da API definido para: {API_BASE_URL}")
print("Headers da requisição configurados.")

Endpoint da API definido para: https://generativelanguage.googleapis.com/v1/models/gemini-1.5-flash:generateContent
Headers da requisição configurados.


In [6]:
# --- Passo 3: Definir o Prompt e a Estrutura da Requisição (JSON Payload) ---

def build_request_payload(user_term):
    """
    Monta o corpo da requisição HTTP em formato JSON para a API Gemini.

    Args:
        user_term (str): O termo de biologia que o usuário digitou.

    Returns:
        dict: O dicionário Python pronto para ser convertido em JSON.
    """
    # O prompt que guia a IA
    prompt_text = f"""
    Você é um assistente especializado em Biologia, focado em explicar conceitos para profissionais da área de Tecnologia da Informação, Análise de Dados e Bioinformática que não possuem conhecimento prévio em Biologia. Sua missão é simplificar termos complexos.

    Para o termo "{user_term}", faça o seguinte:
    1. Forneça uma **explicação objetiva** do que é (gene, proteína, organismo, processo, etc.).
    2. Crie uma **analogia simples** usando conceitos da área de TI, Programação ou Análise de Dados para ajudar a entender a função ou estrutura, quando nao for possivel realizar analogia com coisas cotidianas.
        # >>> MUDE A PRÓXIMA LINHA <<<
    3. Crie uma seção chamada "**Mais informações detalhadas:**" e nela gere 1 ou 2 exemplos de URLs (não precisam ser links ativos, apenas exemplos de formato) para bancos de dados científicos comuns (Ex: NCBI Gene, UniProt, etc.) onde a pessoa poderia pesquisar mais sobre "{user_term}". Use o nome do termo na URL de exemplo se apropriado para a base.

    Formate sua resposta de forma clara, usando talvez tópicos para cada parte.

    Termo a ser explicado: {user_term}
    """

    # Estrutura do corpo da requisição JSON
    # Consulte a documentação da API de generateContent para todos os parâmetros possíveis
    payload = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt_text # prompt está no campo 'text'
                    }
                ]
            }
        ],
        "generationConfig": {
             "temperature": 0.7, # Controla a aleatoriedade (0.0 é mais determinístico)
             "maxOutputTokens": 800 # Limita o tamanho da resposta para não gerar texto excessivo
             # Pode adicionar outras configurações de segurança ou parada aqui se necessário
        },
        # Se estivéssemos usando Function Calling, a descrição da função iria aqui na seção 'tools'
    }

    return payload

print("Função 'build_request_payload' definida.")
print("O prompt para a IA foi configurado com instruções para explicação, analogia TI e links de exemplo.")

Função 'build_request_payload' definida.
O prompt para a IA foi configurado com instruções para explicação, analogia TI e links de exemplo.


In [7]:
# --- Passo 4: Função para Enviar Requisição e Processar Resposta ---

def get_gemini_explanation(user_term, api_url, headers):
    """
    Envia a requisição HTTP para a API Gemini e retorna o texto gerado.

    Args:
        user_term (str): O termo de biologia digitado pelo usuário.
        api_url (str): A URL do endpoint da API Gemini.
        headers (dict): Os cabeçalhos da requisição (incluindo a API Key).

    Returns:
        str: O texto da explicação gerado pela IA, ou uma mensagem de erro.
    """
    # Monta o corpo da requisição usando a função definida na Célula 3
    request_payload = build_request_payload(user_term)

    try:
        # Converte o dicionário Python para uma string JSON
        json_payload = json.dumps(request_payload)

        # Faz a requisição HTTP POST para a API
        # O parâmetro json=request_payload faz a conversão para JSON automaticamente e define Content-Type
        # Content-Type nos headers, então usar 'data' com json.dumps é mais explícito
        # use requests.post(api_url, headers=headers, json=request_payload)
        response = requests.post(api_url, headers=headers, data=json_payload) # data= espera uma string, então usamos json.dumps

        # Verifica se a requisição foi bem-sucedida (código 2xx)
        if response.status_code == 200:
            # Converte a resposta JSON para um dicionário Python
            response_data = response.json()

            # Extrai o texto gerado pela IA
            # A estrutura da resposta JSON é aninhada
            # Verifica se a resposta contém candidatos e partes de texto
            if (response_data and
                'candidates' in response_data and
                len(response_data['candidates']) > 0 and
                'content' in response_data['candidates'][0] and
                'parts' in response_data['candidates'][0]['content'] and
                len(response_data['candidates'][0]['content']['parts']) > 0 and
                'text' in response_data['candidates'][0]['content']['parts'][0]):

                generated_text = response_data['candidates'][0]['content']['parts'][0]['text']
                return generated_text
            else:
                # Caso a estrutura da resposta não seja a esperada ou esteja vazia
                print("\nAviso: Resposta da API não contém texto gerado no formato esperado.")
                print("Resposta completa da API:")
                print(json.dumps(response_data, indent=2)) # Imprime a resposta completa para depuração
                return "Erro: A IA não gerou uma resposta de texto válida."

        else:
            # Se a requisição HTTP não foi bem-sucedida (ex: 400, 401, 404, 500)
            print(f"\nErro na requisição HTTP: Código de Status {response.status_code}")
            print(f"Mensagem de erro da API: {response.text}")
            # Códigos comuns: 400 Bad Request (erro na sua requisição/prompt), 401 Unauthorized (API Key inválida), 403 Forbidden (acesso negado), 500 Erro interno da API
            if response.status_code == 401 or response.status_code == 403:
                 return "Erro: Problema de autenticação com a API Key. Verifique se está correta e ativa."
            else:
                 return f"Erro na API: {response.status_code} - {response.text}"

    except requests.exceptions.RequestException as e:
        # Trata erros na comunicação, como problemas de conexão
        print(f"\nErro de conexão ou requisição: {e}")
        return f"Erro de conexão: Não foi possível se comunicar com a API. Detalhes: {e}"
    except Exception as e:
        # Trata outros erros inesperados
        print(f"\nOcorreu um erro inesperado: {e}")
        return f"Erro inesperado: {e}"


print("Função 'get_gemini_explanation' definida.")

Função 'get_gemini_explanation' definida.


In [ ]:
# --- Passo 5: Loop de Interação com o Usuário ---
print("\n--- Bem vindo ao BioBridge Tech, a ponte entre Biologia e Bioinformatas ---")
print()
print("Você digita o termo de biologia que tem dúvida (gene, proteína, organismo, etc.) e recebe uma explicação objetiva e com analogia de fácil, compreensão, sugestões de análise e link para informações completas.")
print()
print("Para encerrar a consulta digite 'sair'.")

# Certifica-se que a API_KEY foi configurada na Célula 1 antes de rodar este loop
if not API_KEY:
    print("\nAPI Key não configurada. Por favor, execute a Célula 1.")
else:
    while True:
        user_term = input("\n Explique o que é: ")
        if user_term.lower() == 'sair':
            break

        if not user_term.strip(): # Verifica se o input não está vazio
            print("Por favor, digite um termo válido.")
            continue
        print()
        print("-------------------------------")

        print(f"Buscando explicação para '{user_term}'...")
        print()

        # Chama a função que interage com a API
        explanation = get_gemini_explanation(user_term, API_BASE_URL, HEADERS)

        # Exibe a resposta (seja a explicação da IA ou uma mensagem de erro)
        print("\n--- Resposta do Assistente IA ---")
        print()
        print(explanation)
        print("-------------------------------")

print("\n Consulta encerrada.")